In [ ]:
try:
    import napari
    from napari.layers import Shapes
    from napari.utils.notifications import show_info
except:
    pass
import pandas as pd
import random
import tifffile as tiff

import dask.array as da
from dask.cache import Cache
import zarr
import os
import matplotlib.patches as mpatches
import numpy as np
import scipy.spatial.distance as sdistance
#from joblib import Parallel, delayed'
from magicgui import magicgui
import ast
from pathlib import Path

In [ ]:
viewer = napari.Viewer()

In [ ]:
@magicgui(call_button='Open image', layout='vertical')
def open_large_image(image_path=Path(), contrast_limit_txt=Path()):
    #ab = pd.read_csv("D://2022-06-20_Alfredos_project//template//cycle_1to7_renaming.txt")
    #ab = list(ab["ABS"])
    cl_txt = str(contrast_limit_txt)
    
    if cl_txt =='.':
        image = tiff.TiffFile(image_path, is_ome=False) #is_ome=False
        z = zarr.open(image.aszarr(), mode='r') # convert image to Zarr array
        # Identify the number of pyramids
        n_levels = len(image.series[0].levels) # pyramid

        # If and if not pyramids are available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False  

        viewer.add_image(
        pyramid, multiscale=multiscale, channel_axis=0,visible=False,)#name=ab

    else:
        cl = open(cl_txt, 'r')
        c = cl.read()
        contrast_limit = ast.literal_eval(c)
        
        image = tiff.TiffFile(image_path, is_ome=False) #is_ome=False
        z = zarr.open(image.aszarr(), mode='r') # convert image to Zarr array
        # Identify the number of pyramids
        n_levels = len(image.series[0].levels) # pyramid

        # If and if not pyramids are available
        if n_levels > 1:
            pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
            multiscale = True
        else:
            pyramid = da.from_zarr(z)
            multiscale = False  

        viewer.add_image(
        pyramid, multiscale=multiscale, channel_axis=0,visible=False,contrast_limits=contrast_limit,name=ab)
        cl.close()
        
viewer.window.add_dock_widget(open_large_image)


In [ ]:
@magicgui(call_button='Save contrast limits', layout='vertical', output_file={"mode": "w"})
def save_contrast_limits(output_file=Path()):
    contrast_limit = list()
    ab = pd.read_csv("L:\ltdk_farkkila//Data//CyCIF//2023-02-08_S139_geomx//templates//cycle_1to9_renaming.txt")
    ab = list(ab["ABS"])
    print(ab)
    for antibody in ab:
        contrast_limit.append(viewer.layers[antibody].contrast_limits)

    with open(output_file, "w") as output:
        output.write(str(contrast_limit))

viewer.window.add_dock_widget(save_contrast_limits)

In [ ]:
@magicgui(call_button='Save removable cells to csv', filepath={"mode": "w", "mode": "d"})  
def save_list(filepath: Path):
    dict = {'cellid': removable_cells}
    df = pd.DataFrame(dict)
    df = df.astype(int)
    df.to_csv(str(filepath) + viewer.active_layer.name + '_cellids.csv', index=False)

viewer.window.add_dock_widget(save_list)

TAKE SNAPSHOTS

In [ ]:
#ULTIMATE BLENDING - SHAPES MUST BE SQUARES :) - SHAPES LAYER SHOULD BE VISIBLE AND NAMED "Shapes", LAYERS TO SCREENSHOT MUST BE VISIBLE

from PIL import Image
def blended_img(viewer,path):
    blended = None
    layer_rectangle = viewer.layers["Shapes"]
    for i in range(len(layer_rectangle.data)):
        print(i)
        w = int(layer_rectangle.data[i][3][0]) - int(layer_rectangle.data[i][0][0])
        h = int(layer_rectangle.data[i][2][1]) - int(layer_rectangle.data[i][0][1])
        print(w)
        #define blended
        blended = np.zeros((w,h,4))
        # 1. (initial blended_img) iterate over layers to create new transformed layer only with those coordinates
        iterated_layers = []
        for layer in viewer.layers:
            if layer.visible and not isinstance(layer, napari.layers.Shapes):
                contrast_limits = layer.contrast_limits
                colormap = layer.colormap
                name = "transformed" + layer.name
                layer = np.array(layer.data[0][int(layer_rectangle.data[i][0][0]):int(layer_rectangle.data[i][3][0]), int(layer_rectangle.data[i][0][1]):int(layer_rectangle.data[i][2][1])])
                viewer.add_image(layer, name=name ,contrast_limits=contrast_limits,colormap=colormap,visible=False)
                iterated_layers.append(name)
        # 2. iterate over the transformed layers to then create a composite
        blended = np.zeros((w,h,4))
        for layer in viewer.layers:
            if "transformed" in layer.name:
                normalized_data = (layer.data - layer.contrast_limits[0]) / (layer.contrast_limits[1] - layer.contrast_limits[0])
                colormapped_data = layer.colormap.map(normalized_data.flatten())
                colormapped_data = colormapped_data.reshape(normalized_data.shape + (4,))
                blended = blended + colormapped_data
        final_name = str(i)
        extension_name = path + final_name + ".jpg"
        blended[..., 3] = 1 # Set alpha channel to 1 outside of ma
        blended[blended>1]=1
        plt.imsave(extension_name, blended)
        # 3. close the non-utilized windows
        for item in iterated_layers:
            viewer.layers.remove(item)
        print(iterated_layers)
        print("done closing")
    #return blended 

In [ ]:
path="E://snapshots//Post-NACT//IBApos_CD8neg//vim//"
blended_img(viewer,path)

In [ ]:
layer_image = viewer.add_image(img, name='blended')
viewer.layers.remove("blended")

MASK CUT - IN CASE IT IS NEEDED

In [ ]:
@magicgui(call_button='Cut Mask')
def cut_mask():
    mask_to_cut = viewer.layers.selection.active.data
    newmask = np.copy(mask_to_cut)
    shape = viewer.layers.selection.active.data.shape
    selected_area = viewer.layers['Shapes'].to_labels(labels_shape=shape)
    global removable_cells
    removable_cells =[]

    for i in range(0,mask_to_cut.shape[0]):
        for j in range(0,mask_to_cut.shape[1]):
            cell = mask_to_cut[i,j]
            if selected_area[i,j] > 0 and cell not in removable_cells and cell > 0:
                removable_cells.append(cell)
                print(cell)

    return(removable_cells)

@magicgui(call_button='Save removable cells to csv', filepath={"mode": "w", "mode": "d"})  
def save_list(filepath: Path):
    dict = {'cellid': removable_cells}
    df = pd.DataFrame(dict)
    df = df.astype(int)
    df.to_csv(str(filepath) + '/' + viewer.layers.selection.active.name + '_selected_cellids.csv', index=False)

viewer.window.add_dock_widget(cut_mask)
viewer.window.add_dock_widget(save_list)